In [69]:
import numpy as np
import pandas as pd
# import pprint
# pp = pprint.PrettyPrinter(indent=4)


pd.set_option("display.precision", 2)

# some imports to set up plotting
import matplotlib.pyplot as plt

# !pip install seaborn
import seaborn as sns

# import some nice vis settings
sns.set()
# Graphics in the Retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [70]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [71]:
DATA_URL = "./data/"
df_train = pd.read_csv(DATA_URL + "adult_train.csv",  \
                        sep=";", \
                        skipinitialspace=True)

df_test = pd.read_csv(DATA_URL + "adult_test.csv",  \
                        sep=";", \
                        skipinitialspace=True)


In [72]:
df_train.shape

(32561, 15)

In [73]:
df_test.shape

(16282, 15)

Classify people using demographic data - whether they earn more than $50,000 per year or not??

In [74]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Age             32561 non-null  int64 
 1   Workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   Education       32561 non-null  object
 4   Education_Num   32561 non-null  int64 
 5   Martial_Status  32561 non-null  object
 6   Occupation      30718 non-null  object
 7   Relationship    32561 non-null  object
 8   Race            32561 non-null  object
 9   Sex             32561 non-null  object
 10  Capital_Gain    32561 non-null  int64 
 11  Capital_Loss    32561 non-null  int64 
 12  Hours_per_week  32561 non-null  int64 
 13  Country         31978 non-null  object
 14  Target          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [75]:
df_test.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
16277,39,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,0.0,0.0,36.0,United-States,<=50K.
16278,64,NaN,321403.0,HS-grad,9.0,Widowed,NaN,Other-relative,Black,Male,0.0,0.0,40.0,United-States,<=50K.
16279,38,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
16280,44,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455.0,0.0,40.0,United-States,<=50K.
16281,35,Self-emp-inc,182148.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,60.0,United-States,>50K.


In [76]:

df_train.loc[df_train["Education"] == "Bachelors"].shape
#df_train[df_train["Education"] == "Bachelors"].shape


(5355, 15)

In [77]:
# necessary to remove rows with incorrect labels in test dataset
df_test = df_test[
    (df_test["Target"] == ">50K.") | (df_test["Target"] == "<=50K.")
]


# encode target variable as integer
df_train.loc[df_train["Target"] == " <=50K", "Target"] = 0
df_train.loc[df_train["Target"] == " >50K", "Target"] = 1

df_test.loc[df_test["Target"] == " <=50K.", "Target"] = 0
df_test.loc[df_test["Target"] == " >50K.", "Target"] = 1

In [78]:
df_test.shape

(16281, 15)

In [79]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
fnlwgt,16281.0,189435.68,105714.91,13492.0,116736.0,177831.0,238384.0,1.49e+06
Education_Num,16281.0,10.07,2.57,1.0,9.0,10.0,12.0,1.60e+01
Capital_Gain,16281.0,1081.91,7583.94,0.0,0.0,0.0,0.0,1.00e+05
Capital_Loss,16281.0,87.90,403.11,0.0,0.0,0.0,0.0,3.77e+03
Hours_per_week,16281.0,40.39,12.48,1.0,40.0,40.0,45.0,9.90e+01
